L'objectif ici est de convertir chacun de nos models avec Openvino afin que l'inférence soit plus rapide

### Imports

In [1]:
import time
from pathlib import Path
#import cv2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown
from openvino.runtime import Core



### Model 1 : Car or Not

In [2]:
#If you have a model in the HDF5 format, load the model using TensorFlow* 2
# and serialize it in the SavedModel format.

import tensorflow as tf
model = tf.keras.models.load_model('static/FRS_vgg.h5')
tf.keras.models.save_model(model,'static/FRS_vgg')

2022-09-12 03:20:42.449712: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-12 03:20:42.449737: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-12 03:20:46.196550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-12 03:20:46.196577: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-12 03:20:46.196619: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (liganium-HP): /proc/driver/nvidia/version does not exist
2022-09-12 03:20:46.196795: I tensorflow/core/platform/cpu_featur

INFO:tensorflow:Assets written to: static/FRS_vgg/assets


In [3]:
#! mo --saved_model_dir static/carORnot_vgg19

In [4]:
# The paths of the source and converted models.
model_path = Path("static/FRS_vgg")
ir_path = Path("static_openvino/FRS_vgg")

In [5]:
print(ir_path)

static_openvino/FRS_vgg


Convert a TensorFlow Model to OpenVINO IR Format

In [6]:
# Construct the command for Model Optimizer.
# NB : on ne peut spécifier --input_model et --saved_model_dir en même temps
mo_command = f"""mo
                 --saved_model_dir "{model_path}"
                 --input_shape "[1,224,224,3]" 
                 
                 --framework tf
                 --data_type FP16 
                 --output_dir "{ir_path}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert TensorFlow to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert TensorFlow to OpenVINO:


`mo --saved_model_dir "static/FRS_vgg" --input_shape "[1,224,224,3]" --framework tf --data_type FP16 --output_dir "static_openvino/FRS_vgg"`

In [7]:
# Run Model Optimizer if the IR model file does not exist
if not ir_path.exists():
    print("Exporting TensorFlow model to IR... This may take a few minutes.")
    ! $mo_command
else:
    print(f"IR model {ir_path} already exists.")

Exporting TensorFlow model to IR... This may take a few minutes.
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	None
	- Path for generated IR: 	/home/liganium/Documents/Car Damage toolkit/Approche 1 Avec features_extraction et reglogistic/static_openvino/FRS_vgg
	- IR output name: 	saved_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,224,224,3]
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None
Advanced parameters:
	- Force the usage of le

### Test Inference on the Converted Model

### Load the Model

In [8]:
ie = Core()
model = ie.read_model(model=str(ir_path) + "/saved_model.xml", weights=str(ir_path )+ "/saved_model.bin")
compiled_model = ie.compile_model(model=model, device_name="CPU")

### Get Model Information

In [9]:
input_key = compiled_model.input(0)
output_key = compiled_model.output(0)
network_input_shape = input_key.shape

print(input_key)
print(output_key)
print(network_input_shape)

<ConstOutput: names[input_1:0, Func/StatefulPartitionedCall/input/_0:0, input_1] shape{1,224,224,3} type: f32>
<ConstOutput: names[StatefulPartitionedCall/model/dense_2/Softmax:0, StatefulPartitionedCall/Identity:0, Func/StatefulPartitionedCall/output/_39:0, Identity:0] shape{1,3} type: f32>
{1, 224, 224, 3}


### Load an Image

Load an image, resize it, and convert it to the input shape of the network.

In [10]:
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.vgg19 import  preprocess_input

In [11]:
import tensorflow as tf
print(tf.version.VERSION)

2.7.0


In [12]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x



In [13]:
img_path = "static/7.jpg"
img = prepare_img_224(img_path)
img.shape

(1, 224, 224, 3)

### Do Inference

In [15]:
result = compiled_model([img])[output_key]

result_index = np.argmax(result)

print(result)
print(result_index)

[[0. 1. 0.]]
1


In [16]:
labels = ['FRONT','REAR','SIDE']
labels[result_index]

'REAR'